### Approximating the integral

It is easy to see that as we use large $n$ (i.e., more qubits) and thus larger $b_{\mbox{max}}$, we can approximate the integral better as illustrated below. 

In [1]:
import math

b_max = math.pi / 100  # upper limit of integral # 5
nbit = 10  # change this value to get discretized result closer to analytical results # 3

analyticResult = (b_max / 2.0 - math.sin(2 * b_max) / 4.0 ) / b_max  # the target integral can be analytically solved
print("Analytical Result:", analyticResult)

ndiv = 2**nbit  #number of discretization 
discretizedResult = 0.0
for i in range(ndiv):
    discretizedResult += math.sin(b_max / ndiv * (i + 0.5))**2
discretizedResult = discretizedResult / ndiv
print("Discretized Result:", discretizedResult)

Analytical Result: 0.00032892188007938847
Discretized Result: 0.0003289218016943769


In [2]:
import sys
sys.path.append('../../')

In [3]:
#Preparing qiskit environment
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit import IBMQ
from qiskit import Aer
from scipy import optimize
import sys, time
import mpmath as mp
import numpy as np
import matplotlib.pyplot as plt
import pickle
np.random.seed(42)
seed = 42

In [4]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
import qiskit.ignis.mitigation as mit
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter
from lib_grover import *

## Testing the algorithm with simulators

Let us run the algorithm with simulators

In [5]:
#setting the number of shots and Grover operators.
max_iter = 100
# number_grover_list = [0, 1, 2, 4, 8, 16, 32] # , 64, 128, 256]  # list of number of Grover operators
number_grover_list = [5 * i for i in range(20)]
shots_list = [8192] * len(number_grover_list) # list of number of shots
if len(shots_list) != len(number_grover_list):
    raise Exception(
        'The length of shots_list should be equal to the length of number_grover_list.'
    )

backend = Aer.get_backend('qasm_simulator')

In [6]:
noise_model = NoiseModel()
for qi in range(nbit+1):
    if qi >= 0:
        read_err = ReadoutError([[0.950, 0.050],[0.050,0.950]])
        noise_model.add_readout_error(read_err, [qi])

Finally, we can run the algorithm as below: 

In [7]:
qc_list = create_grover_circuit(number_grover_list, nbit,b_max)  # list of Grover circuits

In [8]:
t1 = time.time()
counts_lists = []
for i in range(max_iter): # set the seed as index i
    print("iter", i, "started")
    counts_list = run_grover(qc_list, number_grover_list, shots_list, backend, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)  # list of number of grover operators
    counts_lists.append(counts_list)
    with open("pkls/hists_shots8192.pkl", "wb") as f:
        pickle.dump(counts_lists, f)
    print("iter", i, "finished")
    print()
t2 = time.time()
print(t2-t1)

iter 0 started
0 th round with 0 oracles
0.17689895629882812 s
1 th round with 5 oracles
0.4953601360321045 s
2 th round with 10 oracles
0.8844127655029297 s
3 th round with 15 oracles
1.2497799396514893 s
4 th round with 20 oracles
1.5863890647888184 s
5 th round with 25 oracles
2.1892571449279785 s
6 th round with 30 oracles
2.6167149543762207 s
7 th round with 35 oracles
2.7746918201446533 s
8 th round with 40 oracles
3.021851062774658 s
9 th round with 45 oracles
3.7451000213623047 s
10 th round with 50 oracles
4.058759927749634 s
11 th round with 55 oracles
4.756734132766724 s
12 th round with 60 oracles
5.080130100250244 s
13 th round with 65 oracles
5.35651421546936 s
14 th round with 70 oracles
6.471000909805298 s
15 th round with 75 oracles
6.867233037948608 s
16 th round with 80 oracles
6.134348154067993 s
17 th round with 85 oracles
6.634176015853882 s
18 th round with 90 oracles
8.047839164733887 s
19 th round with 95 oracles
7.908155918121338 s
iter 0 finished

iter 1 star

2.2900490760803223 s
7 th round with 35 oracles
2.6986379623413086 s
8 th round with 40 oracles
3.2537729740142822 s
9 th round with 45 oracles
3.533176898956299 s
10 th round with 50 oracles
3.9594602584838867 s
11 th round with 55 oracles
4.492476224899292 s
12 th round with 60 oracles
4.856084108352661 s
13 th round with 65 oracles
5.42207407951355 s
14 th round with 70 oracles
6.339016914367676 s
15 th round with 75 oracles
5.799037933349609 s
16 th round with 80 oracles
7.006682872772217 s
17 th round with 85 oracles
6.9827210903167725 s
18 th round with 90 oracles
6.821940898895264 s
19 th round with 95 oracles
7.463969707489014 s
iter 8 finished

iter 9 started
0 th round with 0 oracles
0.11105179786682129 s
1 th round with 5 oracles
0.4593970775604248 s
2 th round with 10 oracles
0.8152120113372803 s
3 th round with 15 oracles
1.2001051902770996 s
4 th round with 20 oracles
1.5624299049377441 s
5 th round with 25 oracles
2.1595957279205322 s
6 th round with 30 oracles
2.3621900

4.733707666397095 s
13 th round with 65 oracles
4.950210094451904 s
14 th round with 70 oracles
5.425059080123901 s
15 th round with 75 oracles
5.679629802703857 s
16 th round with 80 oracles
6.183693885803223 s
17 th round with 85 oracles
6.556607007980347 s
18 th round with 90 oracles
6.9173808097839355 s
19 th round with 95 oracles
7.19366717338562 s
iter 16 finished

iter 17 started
0 th round with 0 oracles
0.11056089401245117 s
1 th round with 5 oracles
0.46828603744506836 s
2 th round with 10 oracles
0.8566157817840576 s
3 th round with 15 oracles
1.3085787296295166 s
4 th round with 20 oracles
1.5961840152740479 s
5 th round with 25 oracles
1.9637823104858398 s
6 th round with 30 oracles
2.319981098175049 s
7 th round with 35 oracles
2.687845230102539 s
8 th round with 40 oracles
3.194369077682495 s
9 th round with 45 oracles
3.4721691608428955 s
10 th round with 50 oracles
3.8513588905334473 s
11 th round with 55 oracles
4.361942768096924 s
12 th round with 60 oracles
4.592665

6.913777828216553 s
19 th round with 95 oracles
7.164603233337402 s
iter 24 finished

iter 25 started
0 th round with 0 oracles
0.10895609855651855 s
1 th round with 5 oracles
0.4686291217803955 s
2 th round with 10 oracles
0.8642151355743408 s
3 th round with 15 oracles
1.209549903869629 s
4 th round with 20 oracles
1.7000069618225098 s
5 th round with 25 oracles
1.969933032989502 s
6 th round with 30 oracles
2.314687967300415 s
7 th round with 35 oracles
2.7231948375701904 s
8 th round with 40 oracles
3.192774772644043 s
9 th round with 45 oracles
3.4481990337371826 s
10 th round with 50 oracles
3.8622148036956787 s
11 th round with 55 oracles
4.333326101303101 s
12 th round with 60 oracles
4.53389310836792 s
13 th round with 65 oracles
5.0366551876068115 s
14 th round with 70 oracles
5.329108953475952 s
15 th round with 75 oracles
5.8199028968811035 s
16 th round with 80 oracles
6.180876731872559 s
17 th round with 85 oracles
6.4337317943573 s
18 th round with 90 oracles
6.916359901

1.187885046005249 s
4 th round with 20 oracles
1.5272390842437744 s
5 th round with 25 oracles
2.0360870361328125 s
6 th round with 30 oracles
2.2340710163116455 s
7 th round with 35 oracles
2.5829758644104004 s
8 th round with 40 oracles
2.9963009357452393 s
9 th round with 45 oracles
3.4126009941101074 s
10 th round with 50 oracles
3.6824538707733154 s
11 th round with 55 oracles
4.170431137084961 s
12 th round with 60 oracles
4.451791048049927 s
13 th round with 65 oracles
4.954277276992798 s
14 th round with 70 oracles
5.120373010635376 s
15 th round with 75 oracles
5.628756999969482 s
16 th round with 80 oracles
6.112351894378662 s
17 th round with 85 oracles
6.1864659786224365 s
18 th round with 90 oracles
6.753157138824463 s
19 th round with 95 oracles
7.257501125335693 s
iter 33 finished

iter 34 started
0 th round with 0 oracles
0.10486984252929688 s
1 th round with 5 oracles
0.4605979919433594 s
2 th round with 10 oracles
0.8280911445617676 s
3 th round with 15 oracles
1.1717

...

2.264228105545044 s
7 th round with 35 oracles
2.6192808151245117 s
8 th round with 40 oracles
3.081058979034424 s
9 th round with 45 oracles
3.371429920196533 s
10 th round with 50 oracles
3.780259847640991 s
11 th round with 55 oracles
4.019259214401245 s
12 th round with 60 oracles
4.510240077972412 s
13 th round with 65 oracles
4.817595958709717 s
14 th round with 70 oracles
5.323678970336914 s
15 th round with 75 oracles
5.509428024291992 s
16 th round with 80 oracles
6.031286001205444 s
17 th round with 85 oracles
6.371588706970215 s
18 th round with 90 oracles
6.595484018325806 s
19 th round with 95 oracles
7.1372621059417725 s
iter 66 finished

iter 67 started
0 th round with 0 oracles
0.10513997077941895 s
1 th round with 5 oracles
0.4629700183868408 s
2 th round with 10 oracles
0.8338329792022705 s
3 th round with 15 oracles
1.1919488906860352 s
4 th round with 20 oracles
1.5569860935211182 s
5 th round with 25 oracles
1.948014259338379 s
6 th round with 30 oracles
2.39184617

4.4589457511901855 s
13 th round with 65 oracles
4.899367809295654 s
14 th round with 70 oracles
5.120439052581787 s
15 th round with 75 oracles
5.616863965988159 s
16 th round with 80 oracles
5.848556756973267 s
17 th round with 85 oracles
6.364946126937866 s
18 th round with 90 oracles
6.777419090270996 s
19 th round with 95 oracles
7.0144031047821045 s
iter 74 finished

iter 75 started
0 th round with 0 oracles
0.1064608097076416 s
1 th round with 5 oracles
0.45806384086608887 s
2 th round with 10 oracles
0.8069207668304443 s
3 th round with 15 oracles
1.2024872303009033 s
4 th round with 20 oracles
1.5370151996612549 s
5 th round with 25 oracles
1.9298181533813477 s
6 th round with 30 oracles
2.2765400409698486 s
7 th round with 35 oracles
2.776793956756592 s
8 th round with 40 oracles
3.1019890308380127 s
9 th round with 45 oracles
3.3428940773010254 s
10 th round with 50 oracles
3.8163487911224365 s
11 th round with 55 oracles
4.02443790435791 s
12 th round with 60 oracles
4.5797

6.623301982879639 s
19 th round with 95 oracles
7.006989002227783 s
iter 82 finished

iter 83 started
0 th round with 0 oracles
0.10451412200927734 s
1 th round with 5 oracles
0.4576890468597412 s
2 th round with 10 oracles
0.8462281227111816 s
3 th round with 15 oracles
1.1938259601593018 s
4 th round with 20 oracles
1.6752049922943115 s
5 th round with 25 oracles
1.9287879467010498 s
6 th round with 30 oracles
2.269444227218628 s
7 th round with 35 oracles
2.6506388187408447 s
8 th round with 40 oracles
3.259281873703003 s
9 th round with 45 oracles
3.3464198112487793 s
10 th round with 50 oracles
3.6897449493408203 s
11 th round with 55 oracles
4.294939041137695 s
12 th round with 60 oracles
4.432133197784424 s
13 th round with 65 oracles
4.950275897979736 s
14 th round with 70 oracles
5.22172999382019 s
15 th round with 75 oracles
5.619824171066284 s
16 th round with 80 oracles
5.992048025131226 s
17 th round with 85 oracles
6.200432300567627 s
18 th round with 90 oracles
6.6512079

1.1981589794158936 s
4 th round with 20 oracles
1.5834410190582275 s
5 th round with 25 oracles
1.9334659576416016 s
6 th round with 30 oracles
2.2787601947784424 s
7 th round with 35 oracles
2.7534148693084717 s
8 th round with 40 oracles
3.0573341846466064 s
9 th round with 45 oracles
3.36175799369812 s
10 th round with 50 oracles
3.8067188262939453 s
11 th round with 55 oracles
4.070597171783447 s
12 th round with 60 oracles
4.561661958694458 s
13 th round with 65 oracles
4.782980918884277 s
14 th round with 70 oracles
5.295644760131836 s
15 th round with 75 oracles
5.494771957397461 s
16 th round with 80 oracles
5.981671333312988 s
17 th round with 85 oracles
6.339668035507202 s
18 th round with 90 oracles
6.5266759395599365 s
19 th round with 95 oracles
7.066683053970337 s
iter 91 finished

iter 92 started
0 th round with 0 oracles
0.10518383979797363 s
1 th round with 5 oracles
0.45660901069641113 s
2 th round with 10 oracles
0.8209500312805176 s
3 th round with 15 oracles
1.1873

3.329833984375 s
10 th round with 50 oracles
3.8540151119232178 s
11 th round with 55 oracles
4.10837984085083 s
12 th round with 60 oracles
4.602527141571045 s
13 th round with 65 oracles
4.784379959106445 s
14 th round with 70 oracles
5.299263954162598 s
15 th round with 75 oracles
5.515624046325684 s
16 th round with 80 oracles
6.001780986785889 s
17 th round with 85 oracles
6.433923959732056 s
18 th round with 90 oracles
6.636953115463257 s
19 th round with 95 oracles
7.047961950302124 s
iter 99 finished

7284.355751991272


In [9]:
shots = 8192
qr = QuantumRegister(nbit+1)
mit_pattern = [[i] for i in range(nbit+1)]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
meas_fitters = []
for i in range(max_iter):
    job = execute(meas_calibs, backend=Aer.get_backend('qasm_simulator'), shots=shots, noise_model=noise_model, seed_transpiler=i, seed_simulator=i)
    cal_results = job.result()
    meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
    meas_fitters.append(meas_fitter)
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished
iter 10 finished
iter 11 finished
iter 12 finished
iter 13 finished
iter 14 finished
iter 15 finished
iter 16 finished
iter 17 finished
iter 18 finished
iter 19 finished
iter 20 finished
iter 21 finished
iter 22 finished
iter 23 finished
iter 24 finished
iter 25 finished
iter 26 finished
iter 27 finished
iter 28 finished
iter 29 finished
iter 30 finished
iter 31 finished
iter 32 finished
iter 33 finished
iter 34 finished
iter 35 finished
iter 36 finished
iter 37 finished
iter 38 finished
iter 39 finished
iter 40 finished
iter 41 finished
iter 42 finished
iter 43 finished
iter 44 finished
iter 45 finished
iter 46 finished
iter 47 finished
iter 48 finished
iter 49 finished
iter 50 finished
iter 51 finished
iter 52 finished
iter 53 finished
iter 54 finished
iter 55 finished
iter 56 finished
iter 57 finished
iter 58 finished
iter 59

In [10]:
with open("pkls/meas_fitters.pkl", "wb") as f:
    pickle.dump(meas_fitters, f)